# Paradigm 06: Experience Store Research Agent

This notebook implements the **Experience Store** paradigm from the Research Paradigms document.

## Core Concept

The Experience Store enables non-parametric learning through case-based reasoning:
- **Case Storage**: Store successful research trajectories and outcomes
- **Case Retrieval**: Find similar past cases for new queries
- **Strategy Adaptation**: Apply learned strategies to new problems

## Literature Validation

> "DS-Agent, Agent K, AgentRxiv... demonstrate non-parametric learning through experience storage, achieving 21 gold medals in Kaggle competitions through stored strategies." —Feasibility Report

## Technology Stack

- **LLM**: `gpt-5-mini-2025-08-07`
- **Web Search**: Tavily API
- **Tracing**: LangSmith
- **Framework**: LangGraph
- **Case Storage**: In-memory dictionary (MVP)

## 1. Setup and Configuration

In [ ]:
import os
import operator
import asyncio
import hashlib
import json
from pathlib import Path
from typing import List, Annotated, TypedDict, Literal, Optional, Dict, Any
from datetime import datetime

from dotenv import load_dotenv
from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from tavily import TavilyClient

from langgraph.graph import StateGraph, START, END

# Load environment variables
env_path = Path("../.env")
load_dotenv(env_path)

# Configure LangSmith tracing
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "deep_research_new"

print("Environment configured successfully")

In [ ]:
# Initialize LLM and Tavily client
MODEL_NAME = "gpt-5-mini-2025-08-07"
llm = ChatOpenAI(model=MODEL_NAME, temperature=0)
tavily_client = TavilyClient()

# Experience Store Configuration
EXPERIENCE_STORE_PATH = Path("../data/experience_store.json")
MAX_RETRIEVED_CASES = 3  # Number of similar cases to retrieve
SIMILARITY_THRESHOLD = 0.3  # Minimum similarity for case retrieval

print(f"Using model: {MODEL_NAME}")
print(f"Experience store: {EXPERIENCE_STORE_PATH}")
print(f"Max retrieved cases: {MAX_RETRIEVED_CASES}")

## 2. Experience Store Implementation

In [ ]:
class ExperienceCase(BaseModel):
    """A stored experience case."""
    case_id: str = Field(description="Unique case identifier")
    question: str = Field(description="The research question")
    category: str = Field(default="general", description="Question category")
    keywords: List[str] = Field(default_factory=list, description="Key terms")
    successful_strategies: List[str] = Field(default_factory=list, description="What worked")
    search_queries: List[str] = Field(default_factory=list, description="Effective queries")
    report_structure: List[str] = Field(default_factory=list, description="Section structure")
    quality_score: float = Field(default=0.0, description="Final quality score")
    timestamp: str = Field(default="", description="When this case was created")

In [ ]:
class ExperienceStore:
    """Simple in-memory experience store with persistence."""
    
    def __init__(self, store_path: Path):
        self.store_path = store_path
        self.cases: Dict[str, ExperienceCase] = {}
        self._load()
    
    def _load(self):
        """Load cases from disk."""
        if self.store_path.exists():
            try:
                with open(self.store_path, 'r') as f:
                    data = json.load(f)
                    for case_id, case_data in data.items():
                        self.cases[case_id] = ExperienceCase(**case_data)
                print(f"Loaded {len(self.cases)} cases from experience store")
            except Exception as e:
                print(f"Error loading experience store: {e}")
                self.cases = {}
        else:
            print("No existing experience store found. Starting fresh.")
    
    def _save(self):
        """Save cases to disk."""
        try:
            self.store_path.parent.mkdir(parents=True, exist_ok=True)
            with open(self.store_path, 'w') as f:
                data = {k: v.model_dump() for k, v in self.cases.items()}
                json.dump(data, f, indent=2)
        except Exception as e:
            print(f"Error saving experience store: {e}")
    
    def _generate_case_id(self, question: str) -> str:
        """Generate unique case ID from question."""
        return hashlib.md5(question.encode()).hexdigest()[:12]
    
    def _calculate_similarity(self, query_keywords: List[str], case: ExperienceCase) -> float:
        """Calculate keyword-based similarity."""
        if not query_keywords or not case.keywords:
            return 0.0
        
        query_set = set(kw.lower() for kw in query_keywords)
        case_set = set(kw.lower() for kw in case.keywords)
        
        intersection = len(query_set & case_set)
        union = len(query_set | case_set)
        
        return intersection / union if union > 0 else 0.0
    
    def store_case(self, case: ExperienceCase) -> str:
        """Store a new experience case."""
        if not case.case_id:
            case.case_id = self._generate_case_id(case.question)
        if not case.timestamp:
            case.timestamp = datetime.now().isoformat()
        
        self.cases[case.case_id] = case
        self._save()
        
        return case.case_id
    
    def retrieve_similar(self, keywords: List[str], max_cases: int = 3) -> List[ExperienceCase]:
        """Retrieve similar cases based on keywords."""
        if not self.cases:
            return []
        
        scored_cases = []
        for case in self.cases.values():
            similarity = self._calculate_similarity(keywords, case)
            if similarity >= SIMILARITY_THRESHOLD:
                scored_cases.append((similarity, case))
        
        # Sort by similarity and quality score
        scored_cases.sort(key=lambda x: (x[0], x[1].quality_score), reverse=True)
        
        return [case for _, case in scored_cases[:max_cases]]
    
    def get_stats(self) -> dict:
        """Get store statistics."""
        if not self.cases:
            return {"total_cases": 0}
        
        scores = [c.quality_score for c in self.cases.values()]
        return {
            "total_cases": len(self.cases),
            "avg_quality": sum(scores) / len(scores) if scores else 0,
            "categories": list(set(c.category for c in self.cases.values()))
        }

# Initialize global experience store
experience_store = ExperienceStore(EXPERIENCE_STORE_PATH)
print(f"Store stats: {experience_store.get_stats()}")

## 3. State Definitions

In [ ]:
class ExperienceStoreState(TypedDict):
    """State for the Experience Store Research Agent."""
    # Input
    question: str
    
    # Experience retrieval
    query_keywords: List[str]
    retrieved_cases: List[dict]  # Serialized ExperienceCase objects
    adapted_strategy: str
    
    # Research execution
    search_queries: List[str]
    search_results: Annotated[List[str], operator.add]
    source_urls: Annotated[List[str], operator.add]
    
    # Report
    report_structure: List[str]
    final_report: str
    quality_score: float
    
    # Case creation
    successful_strategies: List[str]

## 4. Helper Functions

In [ ]:
def search_web(query: str, max_results: int = 10) -> tuple[List[str], List[str]]:
    """Execute web search using Tavily. Returns (results, urls)."""
    try:
        if len(query) > 400:
            query = query[:400]
        
        response = tavily_client.search(
            query=query,
            max_results=max_results,
            include_answer=True
        )
        
        results = []
        urls = []
        
        if response.get("answer"):
            results.append(f"Summary: {response['answer']}")
        
        for r in response.get("results", []):
            url = r.get('url', '')
            urls.append(url)
            results.append(f"- {r.get('title', 'No title')}: {r.get('content', '')[:500]}... (Source: {url})")
        
        return results, urls
    except Exception as e:
        return [f"Search error: {str(e)}"], []

## 5. Node Functions

In [ ]:
# Prompts
EXTRACT_KEYWORDS_PROMPT = """Extract 5-10 key terms from this research question that would help find similar past research:

Question: {question}

Return ONLY the keywords, one per line. Include:
- Main topic terms
- Domain/field terms
- Key concepts
"""

ADAPT_STRATEGY_PROMPT = """You are adapting successful research strategies from similar past cases.

Current Question: {question}

Similar Past Cases:
{past_cases}

Based on these past successes, recommend:
1. Search query patterns that worked well
2. Report structure that was effective
3. Key strategies to apply

Be specific and actionable.
"""

GENERATE_QUERIES_PROMPT = """Generate search queries for this research question.

Question: {question}

Strategy guidance from past experience:
{strategy}

Generate 5-7 effective search queries. Apply the patterns that worked before.
Return ONLY the search queries, one per line.
"""

WRITE_REPORT_PROMPT = """Write a comprehensive research report.

Question: {question}

Strategy guidance:
{strategy}

Suggested structure:
{structure}

Research Findings:
{findings}

Write a detailed report (1000-1500 words) following the suggested structure.
Include citations and evidence from the research.
"""

EVALUATE_REPORT_PROMPT = """Evaluate this research report on a scale of 1-10.

Question: {question}

Report:
{report}

Rate on:
1. Relevance to question (1-10)
2. Evidence quality (1-10)
3. Coherence (1-10)
4. Completeness (1-10)

Respond with ONLY four numbers separated by commas (e.g., "8, 7, 9, 8").
"""

In [ ]:
async def extract_keywords(state: ExperienceStoreState) -> dict:
    """Extract keywords from the question for case retrieval."""
    question = state["question"]
    
    print(f"\n{'='*60}")
    print(f"Experience Store: Extracting Keywords")
    print(f"{'='*60}")
    
    prompt = EXTRACT_KEYWORDS_PROMPT.format(question=question)
    response = await llm.ainvoke([HumanMessage(content=prompt)])
    
    keywords = [kw.strip() for kw in response.content.split("\n") if kw.strip()]
    print(f"  Keywords: {keywords[:5]}...")
    
    return {
        "query_keywords": keywords
    }

In [ ]:
async def retrieve_cases(state: ExperienceStoreState) -> dict:
    """Retrieve similar cases from experience store."""
    keywords = state.get("query_keywords", [])
    
    print(f"\n--- Retrieving Similar Cases ---")
    
    similar_cases = experience_store.retrieve_similar(keywords, MAX_RETRIEVED_CASES)
    
    if similar_cases:
        print(f"  Found {len(similar_cases)} similar cases:")
        for case in similar_cases:
            print(f"    - {case.question[:50]}... (score: {case.quality_score:.1f})")
    else:
        print("  No similar cases found. Starting fresh.")
    
    # Serialize cases for state
    serialized = [case.model_dump() for case in similar_cases]
    
    return {
        "retrieved_cases": serialized
    }

In [ ]:
async def adapt_strategy(state: ExperienceStoreState) -> dict:
    """Adapt strategies from retrieved cases."""
    question = state["question"]
    retrieved_cases = state.get("retrieved_cases", [])
    
    print(f"\n--- Adapting Strategy ---")
    
    if not retrieved_cases:
        # Default strategy if no cases found
        strategy = "No prior experience available. Use standard research approach."
        structure = ["Introduction", "Background", "Analysis", "Findings", "Conclusion"]
    else:
        # Format past cases for LLM
        cases_text = ""
        for i, case in enumerate(retrieved_cases, 1):
            cases_text += f"\nCase {i}: {case['question'][:100]}...\n"
            cases_text += f"  Strategies: {', '.join(case.get('successful_strategies', [])[:3])}\n"
            cases_text += f"  Structure: {', '.join(case.get('report_structure', [])[:5])}\n"
            cases_text += f"  Quality: {case.get('quality_score', 0):.1f}/10\n"
        
        prompt = ADAPT_STRATEGY_PROMPT.format(
            question=question,
            past_cases=cases_text
        )
        
        response = await llm.ainvoke([HumanMessage(content=prompt)])
        strategy = response.content
        
        # Extract structure from best case
        best_case = max(retrieved_cases, key=lambda c: c.get('quality_score', 0))
        structure = best_case.get('report_structure', ["Introduction", "Analysis", "Conclusion"])
    
    print(f"  Strategy adapted (length: {len(strategy)} chars)")
    
    return {
        "adapted_strategy": strategy,
        "report_structure": structure
    }

In [ ]:
async def execute_research(state: ExperienceStoreState) -> dict:
    """Execute research with adapted strategy."""
    question = state["question"]
    strategy = state.get("adapted_strategy", "")
    
    print(f"\n--- Executing Research ---")
    
    # Generate queries using strategy
    prompt = GENERATE_QUERIES_PROMPT.format(
        question=question,
        strategy=strategy[:500]
    )
    
    response = await llm.ainvoke([HumanMessage(content=prompt)])
    queries = [q.strip() for q in response.content.split("\n") if q.strip()][:7]
    
    # Execute searches
    all_results = []
    all_urls = []
    
    for query in queries:
        print(f"  Searching: {query[:50]}...")
        results, urls = search_web(query)
        all_results.extend(results)
        all_urls.extend(urls)
    
    print(f"  Collected {len(all_results)} results from {len(set(all_urls))} sources")
    
    return {
        "search_queries": queries,
        "search_results": all_results,
        "source_urls": all_urls
    }

In [ ]:
async def write_report(state: ExperienceStoreState) -> dict:
    """Write report using adapted structure."""
    question = state["question"]
    strategy = state.get("adapted_strategy", "")
    structure = state.get("report_structure", [])
    search_results = state.get("search_results", [])
    
    print(f"\n--- Writing Report ---")
    
    prompt = WRITE_REPORT_PROMPT.format(
        question=question,
        strategy=strategy[:500],
        structure="\n".join(f"- {s}" for s in structure),
        findings="\n\n".join(search_results[:25])
    )
    
    response = await llm.ainvoke([HumanMessage(content=prompt)])
    
    print(f"  Report generated: {len(response.content)} chars")
    
    return {
        "final_report": response.content
    }

In [ ]:
async def evaluate_and_store(state: ExperienceStoreState) -> dict:
    """Evaluate report and store experience."""
    question = state["question"]
    report = state.get("final_report", "")
    keywords = state.get("query_keywords", [])
    queries = state.get("search_queries", [])
    structure = state.get("report_structure", [])
    
    print(f"\n--- Evaluating and Storing Experience ---")
    
    # Evaluate report
    eval_prompt = EVALUATE_REPORT_PROMPT.format(
        question=question,
        report=report[:3000]
    )
    
    response = await llm.ainvoke([HumanMessage(content=eval_prompt)])
    
    # Parse scores
    try:
        scores = [float(s.strip()) for s in response.content.split(",")][:4]
        quality_score = sum(scores) / len(scores)
    except:
        quality_score = 6.0
        scores = [6, 6, 6, 6]
    
    print(f"  Quality scores: {scores}")
    print(f"  Average: {quality_score:.1f}/10")
    
    # Identify successful strategies
    strategies = []
    if quality_score >= 7:
        strategies.append("Used comprehensive search queries")
        strategies.append(f"Applied {len(queries)} targeted searches")
        if state.get("retrieved_cases"):
            strategies.append("Adapted strategies from similar past cases")
    
    # Store the case
    case = ExperienceCase(
        case_id="",
        question=question,
        category="research",
        keywords=keywords,
        successful_strategies=strategies,
        search_queries=queries,
        report_structure=structure,
        quality_score=quality_score
    )
    
    case_id = experience_store.store_case(case)
    print(f"  Stored as case: {case_id}")
    print(f"  Total cases in store: {len(experience_store.cases)}")
    
    return {
        "quality_score": quality_score,
        "successful_strategies": strategies
    }

## 6. Graph Construction

In [ ]:
# Build the Experience Store Research Agent graph
es_builder = StateGraph(ExperienceStoreState)

# Add nodes
es_builder.add_node("extract_keywords", extract_keywords)
es_builder.add_node("retrieve_cases", retrieve_cases)
es_builder.add_node("adapt_strategy", adapt_strategy)
es_builder.add_node("execute_research", execute_research)
es_builder.add_node("write_report", write_report)
es_builder.add_node("evaluate_and_store", evaluate_and_store)

# Add edges - linear flow
es_builder.add_edge(START, "extract_keywords")
es_builder.add_edge("extract_keywords", "retrieve_cases")
es_builder.add_edge("retrieve_cases", "adapt_strategy")
es_builder.add_edge("adapt_strategy", "execute_research")
es_builder.add_edge("execute_research", "write_report")
es_builder.add_edge("write_report", "evaluate_and_store")
es_builder.add_edge("evaluate_and_store", END)

# Compile
experience_store_graph = es_builder.compile()

print("Experience Store Research Agent compiled successfully")

In [ ]:
# Visualize the graph
from IPython.display import Image, display

try:
    display(Image(experience_store_graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not display graph: {e}")

## 7. Agent Wrapper for Evaluation

In [ ]:
def experience_store_agent(inputs: dict) -> dict:
    """
    Wrapper function for Experience Store research agent.
    
    Compatible with evaluation harness.
    
    Args:
        inputs: Dictionary with 'question' key
        
    Returns:
        Dictionary with 'output' key containing final report
    """
    question = inputs.get("question", "")
    
    # Run with recursion limit
    result = asyncio.run(
        experience_store_graph.ainvoke(
            {"question": question},
            config={"recursion_limit": 50}
        )
    )
    
    return {
        "output": result.get("final_report", ""),
        "quality_score": result.get("quality_score", 0),
        "retrieved_cases": len(result.get("retrieved_cases", [])),
        "source_urls": result.get("source_urls", [])
    }

## 8. Manual Test

Run this cell to verify the agent works correctly with a simple test question.

In [ ]:
# Simple test
test_question = "What are the key benefits and challenges of using large language models in enterprise applications?"

print(f"Testing Experience Store Agent with question:\n{test_question}\n")
print("Running experience-guided research (this may take several minutes)...\n")

try:
    result = experience_store_agent({"question": test_question})
    
    print("\n" + "=" * 80)
    print("FINAL REPORT")
    print("=" * 80)
    print(result["output"][:3000] + "..." if len(result["output"]) > 3000 else result["output"])
    print("\n" + "=" * 80)
    print(f"Report length: {len(result['output'])} characters")
    print(f"Quality score: {result.get('quality_score', 0):.1f}/10")
    print(f"Similar cases used: {result.get('retrieved_cases', 0)}")
    print(f"Unique sources: {len(set(result.get('source_urls', [])))}")
    print(f"\nExperience Store Stats: {experience_store.get_stats()}")
    print("Agent test PASSED ✓")
except Exception as e:
    print(f"Agent test FAILED: {e}")
    import traceback
    traceback.print_exc()
    raise

## 9. Evaluation Harness Integration

Once the manual test passes, uncomment and run the cells below for full evaluation.

In [ ]:
# Import evaluation harness and metrics
import sys
sys.path.insert(0, "..")
from evaluation import (
    ExperimentHarness, 
    fact_recall, 
    citation_precision,
    coherence_judge, 
    depth_judge, 
    relevance_judge,
    minimum_sources_check
)

# Initialize harness with the golden test dataset
harness = ExperimentHarness(
    dataset_path="../data/deep_research_agent_test_dataset.yaml",
    langsmith_dataset_name="deep-research-golden-v2"
)

print("Evaluation harness initialized successfully!")
print(f"Dataset: {harness.dataset_path}")
print(f"LangSmith dataset name: {harness.langsmith_dataset_name}")

In [ ]:
# Full Evaluation on All 20 Questions
# ⚠️ EXPENSIVE - Only uncomment when ready for full evaluation
# NOTE: The Experience Store improves over time as it accumulates cases.
# Running the full evaluation will populate the store with experiences.
# Uncomment to run:

# # Define comprehensive evaluator suite
# evaluators = [
#     fact_recall,              # Required facts coverage
#     citation_precision,       # Citation URL validity
#     minimum_sources_check,    # Minimum source count
#     coherence_judge,          # Logical structure
#     depth_judge,              # Analysis depth
#     relevance_judge,          # Addresses question
# ]
# 
# # Run full evaluation
# print("Starting FULL evaluation on all 20 questions...")
# print("Experience Store Agent - this will take 1-2 hours.")
# print(f"Starting with {len(experience_store.cases)} existing cases.")
# print("=" * 80 + "\n")
# 
# results = harness.run_evaluation(
#     agent_fn=experience_store_agent,
#     evaluators=evaluators,
#     experiment_name="experience_store_v1",
#     monte_carlo_runs=1,  # Single run to reduce cost
#     max_concurrency=2,   # Lower concurrency for stability
#     description="Experience Store paradigm evaluation on all difficulty tiers"
# )
# 
# # Display comprehensive results
# print("\n" + "=" * 80)
# print("FULL EVALUATION RESULTS")
# print("=" * 80)
# print(f"Experiment: {results.experiment_name}")
# print(f"Questions evaluated: {results.num_questions}")
# print(f"Runs per question: {results.num_runs}")
# print(f"\nExperience Store now contains: {len(experience_store.cases)} cases")
# 
# print(f"\n{'Metric':<30} {'Mean':<10}")
# print("-" * 40)
# for metric_name in sorted(results.metrics.keys()):
#     if not metric_name.endswith('_std'):
#         value = results.metrics.get(metric_name, 0)
#         print(f"{metric_name:<30} {value:<10.3f}")
# 
# # Save results to file
# import json
# from datetime import datetime
# 
# results_file = Path("../results") / f"experience_store_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
# results_file.parent.mkdir(exist_ok=True)
# 
# with open(results_file, 'w') as f:
#     json.dump({
#         "experiment_name": results.experiment_name,
#         "num_questions": results.num_questions,
#         "num_runs": results.num_runs,
#         "metrics": results.metrics,
#         "per_question": results.per_question_results,
#         "final_store_size": len(experience_store.cases)
#     }, f, indent=2)
# 
# print(f"\nResults saved to: {results_file}")

print("Full evaluation cell ready. Uncomment to run when ready.")